In [173]:
import pandas as pd
from gurobipy import *
import numpy as np

In [174]:
# Reading the data from the excel sheets
source_df = pd.read_excel("Source_facility_info.xlsx", sheet_name="PZA")
destination_df = pd.read_excel("Destination_facility_info.xlsx", sheet_name="PZE")
trucking_df = pd.read_excel("Trucking_info.xlsx", sheet_name="Truck")

# Convert relevant columns to datetime and normalize shift times
def normalize_shift_times(row):
    start_hour = row['Start of shift'].hour + row['Start of shift'].minute / 60
    end_hour = row['End of lay-on'].hour + row['End of lay-on'].minute / 60
    if end_hour < start_hour:
        end_hour += 24  # normalize end_hour for the next day
    return pd.Series([start_hour, end_hour])

destination_df[['Start of shift', 'End of lay-on']] = destination_df.apply(normalize_shift_times, axis=1)
source_df['planned_end_of_loading'] = pd.to_datetime(source_df['planned_end_of_loading'])



In [175]:
# Initialize the model
model = Model("DHL_Optimization")

# Decision Variables
source_list = list(trucking_df['Origin_ID'].unique()[:1])  # Index of PZA
destination_list = list(trucking_df['Destination_ID'].unique()[:5])  # Index of PZE
routes_list = [(i, j) for i in source_list for j in destination_list if i != j]
consignment_list = [
    x for (i, j) in routes_list
    for x in source_df[(source_df['Origin_ID'] == i) & (source_df['Destination_ID'] == j)]['id'].values
]

valid_combinations = [(i, j, k) for (i, j) in routes_list for k in consignment_list if k in source_df[(source_df['Origin_ID'] == i) & (source_df['Destination_ID'] == j)]['id'].values]

trucks = range(300)

# Decision Variables
X = {}
Z = {}
T = {}
ArrivalDay = {}
ArrivalTime = {}
ArrivalDayBinary = {}

for (i, j, k, l) in [(i, j, k, l) for (i, j, k) in valid_combinations for l in trucks]:
    X[(i, j, k, l)] = model.addVar(vtype=GRB.BINARY, name=f"X_{i}_{j}_{k}_{l}")

for l in trucks:
    Z[l] = model.addVar(vtype=GRB.BINARY, name=f"Z_{l}")
    T[l] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"T_{l}")
    ArrivalDay[l] = model.addVar(lb = 1, ub=4, vtype=GRB.INTEGER, name=f"ArrivalDay_{l}")
    ArrivalTime[l] = model.addVar(lb=0, ub=24, vtype=GRB.CONTINUOUS, name=f"ArrivalTime_{i}_{j}_{k}_{l}")

    for d in range(1, 5):  # Maximum number of days to consider
        ArrivalDayBinary[(l, d)] = model.addVar(vtype=GRB.BINARY, name=f"ArrivalDayBinary_{l}_{d}")


In [176]:
# Objective function: Minimize the total arrival time
model.setObjective(
    quicksum(X[i, j, k, l] for (i, j, k, l) in [(i, j, k, l) for (i, j, k) in valid_combinations for l in trucks]) 
    - quicksum(Z[l] for l in trucks) 
    - quicksum(ArrivalDay[l] for l in trucks), 
    GRB.MAXIMIZE
)

In [177]:
# Constraints
# 1. Each truck can carry at most 2 consignments
for l in trucks:
    model.addConstr(quicksum(X[(i, j, k, l)] for (i, j, k) in valid_combinations) <= 2 * Z[l])

In [178]:
# 2. Consignment can only be released after the latest release time of the consignments
for (i, j, k) in valid_combinations:
    release_time = source_df[source_df['id'] == k]['planned_end_of_loading'].dt.hour.values[0]  # Convert to hours
    for l in trucks:
        model.addConstr(T[l] >= release_time * X[(i, j, k, l)])

In [179]:
# Constraint 3: Truck must arrive at the destination within the operational hours
for (i, j, k) in valid_combinations:
    start_shift = destination_df[destination_df['Destination_ID'] == j]['Start of shift'].values[0]
    end_shift = destination_df[destination_df['Destination_ID'] == j]['End of lay-on'].values[0]
    travel_time = trucking_df[(trucking_df['Origin_ID'] == i) & (trucking_df['Destination_ID'] == j)]['OSRM_time [sek]'].values[0] / 3600  # Convert to hours
    # print(f"Route {i}->{j}->{k} | Start Shift: {start_shift}, End Shift: {end_shift}, Travel Time: {travel_time}")
    for l in trucks:
        ArrivalTime[(l)] = (T[l] + travel_time * X[(i, j, k, l)] + 24 * ArrivalDay[l]) - 24 * model.addVar(vtype=GRB.INTEGER, name=f"multiplier_{i}_{j}_{k}_{l}")
        model.addConstr(ArrivalTime[(l)] >= start_shift)
        model.addConstr(ArrivalTime[(l)] <= end_shift)

In [180]:
# 4. Each consignment must be assigned to exactly one truck
for (i, j, k) in valid_combinations:
    model.addConstr(quicksum(X[(i, j, k, l)] for l in trucks) == 1)

In [181]:
M=4
for l in trucks:
        for d in range(1, 5): 
                 # Maximum number of days to consider
                model.addConstr(ArrivalDay[l] - d <= M * (1 - ArrivalDayBinary[(l, d)]))
                model.addConstr(ArrivalDay[l] - d >= 1 - M * ArrivalDayBinary[(l, d)])

In [182]:
for j in destination_list:
    working_hours = destination_df[destination_df['Destination_ID'] == j]['End of lay-on'].values[0] - destination_df[destination_df['Destination_ID'] == j]['Start of shift'].values[0]
    sorting_capacity_per_day =  working_hours * destination_df[destination_df['Destination_ID'] == j]['Sorting capacity'].values[0]
    for d in range(1, 5):  # Maximum number of days to consider
        model.addConstr(
            quicksum(X[(i, j, k, l)] * source_df[source_df['id'] == k]['Consignment quantity'].values[0] * ArrivalDayBinary[(l, d)]
                     for i in source_list if j != i
                     for k in consignment_list
                     for l in trucks
                     if (i, j, k) in valid_combinations) <= sorting_capacity_per_day,
            name=f"SortingCapacity_{j}_{d}"
        )


In [183]:
# for l in trucks:
#     model.addConstr(
#         quicksum(ArrivalDayBinary[(l, d)] for d in range(1, 5)) == 1,
#         name = f'Assigning Arrival Day to each truck'
#         )

In [185]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1240P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 87398 rows, 58800 columns and 343500 nonzeros
Model fingerprint: 0x6f4c3cdf
Model has 16 quadratic constraints
Variable types: 600 continuous, 58200 integer (29700 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [4e+02, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 1e+05]
  QRHS range       [1e+05, 4e+05]
Presolved: 171406 rows, 142800 columns, 680700 nonzeros

Continuing optimization...


Explored 1 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 2: -253 -900 
No other solutions better than -253

Optimal solution found (tolerance 1.00e-04)
Best objective 

In [186]:
# Accessing ArrivalDay values after optimization
if model.status in [GRB.OPTIMAL, GRB.TIME_LIMIT, GRB.SUBOPTIMAL]:
    # Extract the data into a DataFrame
    data = []
    k_set = set()
    for (i, j, k, l) in X.keys():
        if (Z[l].X == 1 ) and (X[i,j,k,l].X == 1.0):
            start_shift = destination_df[destination_df['Destination_ID'] == j]['Start of shift'].values[0]
            end_shift = destination_df[destination_df['Destination_ID'] == j]['End of lay-on'].values[0]
            travel_time = trucking_df[(trucking_df['Origin_ID'] == i) & (trucking_df['Destination_ID'] == j)]['OSRM_time [sek]'].values[0] / 3600
            arrival_time = (T[l].X + travel_time + 24 * ArrivalDay[l].X)%24
            data.append({
                'Origin(PZA)': i,
                'Destination (PZE)': j,
                'Consignment ID': k,
                'Truck Id': l,
                'Departure time': T[l].X,
                'Arrival Day': ArrivalDay[l].X,
                'Arrival Time': arrival_time,
                'Arrival Day Binary': ArrivalDayBinary[l, ArrivalDay[l].X].X,
                "Destination Start Shift": start_shift,
                "Destination End Shift": end_shift,
                "Travel Time": travel_time
                
          })

In [187]:
df = pd.DataFrame(data)

In [189]:
import os
path = os.getcwd()
df.to_csv("output/output.csv")

In [16]:
model.write("file.lp")

In [188]:
df.head()

,Origin(PZA),Destination (PZE),Consignment ID,Truck Id,Departure time,Arrival Day,Arrival Time,Arrival Day Binary,Destination Start Shift,Destination End Shift,Travel Time
0,01.1.1.PZ,04.1.1.PZ,7791768,254,22.0,1.0,0.182361,1.0,22.0,30.75,2.182361
1,01.1.1.PZ,04.1.1.PZ,7791769,254,22.0,1.0,0.182361,1.0,22.0,30.75,2.182361
2,01.1.1.PZ,04.1.1.PZ,7791770,253,22.0,1.0,0.182361,1.0,22.0,30.75,2.182361
3,01.1.1.PZ,04.1.1.PZ,7791771,253,22.0,1.0,0.182361,1.0,22.0,30.75,2.182361
4,01.1.1.PZ,04.1.1.PZ,7791772,299,22.0,1.0,0.182361,1.0,22.0,30.75,2.182361
